In [20]:
from pymilvus import MilvusClient, model
from config import OPENAI_API_KEY
import json

DB_PATH = "datas/final_result.db"
OPENAI_TEXT_EMBEDDING_MODEL = "text-embedding-3-small"
EMBEDDING_DIMENSION = 1536
COLLECTION_NAME = "smartstore_faq"


openai_ef = model.dense.OpenAIEmbeddingFunction(
    model_name=OPENAI_TEXT_EMBEDDING_MODEL,
    api_key=OPENAI_API_KEY
)

with open("datas/final_result.json", "r", encoding="utf-8") as f:
    faq_data = json.load(f)

milvus_client = MilvusClient(DB_PATH)


In [17]:
len(faq_data.items())

2717

In [36]:
# create collection
if not milvus_client.has_collection(COLLECTION_NAME):
    milvus_client.create_collection(COLLECTION_NAME, dimension=EMBEDDING_DIMENSION)

In [21]:
from db import openai_ef

# insert data with embeddings
datas = []
for i, (key, value) in enumerate(faq_data.items()):

    string_to_embed = f"{key}: {value}"

    # cut the string for over 8192 tokens
    string_to_embed = string_to_embed[:8000]

    try:
        vector = openai_ef.encode_documents(string_to_embed)[0]
    except Exception as e:
        print(i)
        print(len(string_to_embed))
        raise e
    
    data = {
        "id": i,
        "question": key,
        "answer": value,
        "vector": vector
    }

    milvus_client.insert(COLLECTION_NAME, data)

In [23]:
milvus_client.get_collection_stats(COLLECTION_NAME)

{'row_count': 2717}